## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [14]:
%%sql
postgresql:///animal_shelters

WITH 
-- Create a table without sponsorship --
nonsponsor_animals AS
(SELECT animaltype, weight, location,
DATE_PART('year', AGE('2021-12-31'::timestamp, birthdate::timestamp))::varchar AS age
FROM animals
WHERE animalid NOT IN
(SELECT sponsorid
FROM sponsored_animals)),

-- Creat a dog_table to classify size, sizeid --
dog_table AS
(SELECT *, 
    CASE WHEN weight <= 10 THEN 'small'
    WHEN weight <= 30 THEN 'medium'
    ELSE 'large' END AS size
FROM nonsponsor_animals
WHERE animaltype = 'Dog'),

-- Creat a cat_table to classify size, sizeid --
cat_table AS
(SELECT *, 
    CASE WHEN weight <= 5 THEN 'small'
    WHEN weight <= 7 THEN 'medium'
    ELSE 'large' END AS size
FROM nonsponsor_animals
WHERE animaltype = 'Cat'),

-- Creat a bird_table to classify size, sizeid --
bird_table AS
(SELECT *, 
    CASE WHEN weight <= 0.7 THEN 'small'
    WHEN weight <= 1.1 THEN 'medium'
    ELSE 'large' END AS size
FROM nonsponsor_animals
WHERE animaltype = 'Bird'),


-- Union all tables to get all dataset --
union_all AS
(SELECT *, 
    CASE WHEN size = 'small' THEN 'DS'
    WHEN size = 'medium' THEN 'DM'
    ELSE 'DL' END AS sizeid
FROM dog_table
UNION ALL
SELECT *,
    CASE WHEN size = 'small' THEN 'CS'
    WHEN size = 'medium' THEN 'CM'
    ELSE 'CL' END AS sizeid
FROM cat_table
UNION ALL
SELECT *,
    CASE WHEN size = 'small' THEN 'BS'
    WHEN size = 'medium' THEN 'BM'
    ELSE 'BL' END AS sizeid
FROM bird_table)

-- Join the tables --
SELECT LOWER(u.animaltype) AS animaltype, u.size,
       SUM(l.costs + a.costs + s.costs) AS total,
       ROUND(100*SUM(l.costs + a.costs + s.costs)/SUM(SUM(l.costs + a.costs + s.costs)) OVER(),2) AS percentage
    
FROM union_all AS u

LEFT JOIN location_costs AS l
ON u.location = l.location

LEFT JOIN age_costs AS a
ON a.age = u.age

LEFT JOIN size_costs AS s
ON u.sizeid = s.sizeid
GROUP BY u.animaltype, u.size
ORDER BY u.animaltype, u.size DESC


9 rows affected.


animaltype,size,total,percentage
bird,small,1615,0.05
bird,medium,3130,0.09
bird,large,8120,0.23
cat,small,518015,14.90
cat,medium,250575,7.21
cat,large,439490,12.64
dog,small,336530,9.68
dog,medium,941895,27.09
dog,large,977665,28.12
